In [370]:
import pandas as pd
data = pd.read_csv("../0-Data/mushroom_dataset.csv")
pd.set_option('display.max_columns', len(data.columns))
print(len(data.columns))
data.head()

23


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,mushroom
0,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE
1,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS,EDIBLE
2,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE
3,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS,EDIBLE
4,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,BROWN,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE


In [371]:
data.dtypes

cap-shape                   object
cap-surface                 object
cap-color                   object
bruises                     object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
mushroom                    object
dtype: object

In [372]:
toEncode = [column for column in data.columns if column != 'mushroom']
for i in toEncode:
    encoded = (pd.get_dummies(data[i], dtype=float))
    data = pd.concat([data, encoded], axis=1)
    data.drop(i, axis=1, inplace=True)
pd.set_option("display.max_columns", len(data.columns))
print(len(data.columns))
data

118


,mushroom,BELL,CONICAL,CONVEX,FLAT,KNOBBED,SUNKEN,FIBROUS,GROOVES,SCALY,SMOOTH,BROWN,BUFF,CINNAMON,GRAY,GREEN,PINK,PURPLE,RED,WHITE,YELLOW,BRUISES,NO,ALMOND,ANISE,CREOSOTE,FISHY,FOUL,MUSTY,NONE,PUNGENT,SPICY,ATTACHED,FREE,CLOSE,CROWDED,BROAD,NARROW,BLACK,BROWN,BUFF,CHOCOLATE,GRAY,GREEN,ORANGE,PINK,PURPLE,RED,WHITE,YELLOW,ENLARGING,TAPERING,?,BULBOUS,CLUB,EQUAL,ROOTED,FIBROUS,SCALY,SILKY,SMOOTH,FIBROUS,SCALY,SILKY,SMOOTH,BROWN,BUFF,CINNAMON,GRAY,ORANGE,PINK,RED,WHITE,YELLOW,BROWN,BUFF,CINNAMON,GRAY,ORANGE,PINK,RED,WHITE,YELLOW,PARTIAL,BROWN,ORANGE,WHITE,YELLOW,NONE,ONE,TWO,EVANESCENT,FLARING,LARGE,NONE,PENDANT,BLACK,BROWN,BUFF,CHOCOLATE,GREEN,ORANGE,PURPLE,WHITE,YELLOW,ABUNDANT,CLUSTERED,NUMEROUS,SCATTERED,SEVERAL,SOLITARY,GRASSES,LEAVES,MEADOWS,PATHS,URBAN,WASTE,WOODS
0,EDIBLE,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EDIBLE,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,EDIBLE,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,EDIBLE,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,EDIBLE,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8411,EDIBLE,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,

In [373]:
import numpy as np

corr_matrix = (data.drop('mushroom', axis=1)).corr(method='pearson')

# Criar uma máscara para correlações superiores a 0.75
high_corr_var = np.where(np.abs(corr_matrix) > 0.75)
high_corr_var = [(corr_matrix.index[x], corr_matrix.columns[y]) for x, y in zip(*high_corr_var) if x != y and x < y]

# Identificar as colunas a serem eliminadas
cols_to_drop = set()
for pair in high_corr_var:
    cols_to_drop.add(pair[1])  # Adiciona a segunda coluna do par, pode escolher a primeira se preferir

# Eliminar as colunas com alta correlação
data = data.drop(columns=cols_to_drop)
print(len(data.columns))

74


In [374]:
print(data['mushroom'].value_counts())
data['mushroom'] = data['mushroom'].map({'EDIBLE':0, 'POISONOUS':1})
print(data['mushroom'].value_counts())

mushroom
EDIBLE       4488
POISONOUS    3928
Name: count, dtype: int64
mushroom
0    4488
1    3928
Name: count, dtype: int64


In [375]:
x = data.drop('mushroom', axis=1)
y = data['mushroom']

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import numpy as np

normalizer = MinMaxScaler(feature_range=(0,1))
x_norm = normalizer.fit_transform(x)
pca = PCA(n_components=40)
x_pca = pca.fit_transform(x_norm)

variance = pca.explained_variance_ratio_
print("Explained variance:", variance)
print("Sum variance:", sum(variance))

skfold = StratifiedKFold(n_splits=5, shuffle=True)

param = {
}

model = ExtraTreesClassifier(n_estimators=100, bootstrap=True, n_jobs=-1)
grid_model = GridSearchCV(estimator=model, param_grid=param, cv=skfold, n_jobs=-1)
grid_model.fit(x_pca, y)


print(f"Best samples split:", grid_model.best_estimator_.min_samples_split)
print(f"Best depth: {grid_model.best_estimator_.max_depth}")
print(f"Best samples leaf: {grid_model.best_estimator_.min_samples_leaf}")
print(f"R2: {grid_model.best_score_}")

Explained variance: [0.13507773 0.12599454 0.10564357 0.06739815 0.05758675 0.03479272
 0.03382438 0.03262183 0.02873126 0.02399734 0.02270795 0.02120418
 0.02046999 0.01967705 0.01964729 0.01638564 0.0160479  0.01564669
 0.01452777 0.01340105 0.01291057 0.01241345 0.0103924  0.00963818
 0.00915806 0.00861274 0.00823764 0.00788232 0.00722616 0.00669318]
Sum variance: 0.9185484934930709
Best samples split: 2
Best depth: None
Best samples leaf: 1
R2: 1.0


In [376]:

from sklearn.model_selection import cross_val_score

skfold = StratifiedKFold(n_splits=3, shuffle=True)
result = cross_val_score(model, x_pca, y, cv=skfold, n_jobs=-1)
print(result.mean())

1.0
